In [1]:
import argparse
import torch
import numpy as np
import pandas as pd

from recbole.quick_start import run_recbole, load_data_and_model

In [ ]:
# import sys, os
# # path 설정
# sys.path.append((os.path.abspath("")))
# print(sys.path[-1])

### Train & Valid

In [2]:
parameter_dict = {
    'train_neg_sample_args': None,  # 암시적 피드백 데이터를 위한 설정
    'mode': 'full',
}
run_recbole(config_file_list=['recbole_ease_1.yaml'], config_dict=parameter_dict)

23 Nov 10:50    INFO  ['d:\\Program\\Dev\\ANACONDA\\envs\\movie\\lib\\site-packages\\ipykernel_launcher.py', '--f=c:\\Users\\shgkd\\AppData\\Roaming\\jupyter\\runtime\\kernel-v3e2a25bc3dd1142fffa4339bd5d4d86411ea1daf8.json']
23 Nov 10:50    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = ./data/train\train_ratings
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.15, 0.05]}, 'order': 'RO', 'group_by': 'user', 'mode': {'va

Train     0:   0%|                                                         | 0/2029 [00:00<?, ?it/s…

d:\Program\Dev\ANACONDA\envs\movie\lib\site-packages\recbole\trainer\trainer.py:235: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler(enabled=self.enable_scaler)
23 Nov 10:54    INFO  epoch 0 training [time: 5.29s, train loss: 0.0000]


Evaluate   :   0%|                                                        | 0/31360 [00:00<?, ?it/s…

23 Nov 10:55    INFO  epoch 0 evaluating [time: 43.62s, valid_score: 0.166300]
23 Nov 10:55    INFO  valid result: 
mrr@10 : 0.622    recall@10 : 0.1663    precision@10 : 0.3115
23 Nov 10:55    INFO  Saving current: saved\EASE-Nov-23-2024_10-54-23.pth
d:\Program\Dev\ANACONDA\envs\movie\lib\site-packages\recbole\trainer\trainer.py:583: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`

Evaluate   :   0%|                                                        | 0/31360 [00:00<?, ?it/s…

23 Nov 10:56    INFO  The running environment of this training is as follows:
+-------------+----------------+
| Environment |     Usage      |
+=============+================+
| CPU         |     2.10 %     |
+-------------+----------------+
| GPU         | 0.00 G/6.00 G  |
+-------------+----------------+
| Memory      | 2.29 G/31.67 G |
+-------------+----------------+
23 Nov 10:56    INFO  best valid : OrderedDict([('mrr@10', 0.622), ('recall@10', 0.1663), ('precision@10', 0.3115)])
23 Nov 10:56    INFO  test result: OrderedDict([('mrr@10', 0.3999), ('recall@10', 0.2048), ('precision@10', 0.1351)])


{'best_valid_score': 0.1663,
 'valid_score_bigger': True,
 'best_valid_result': OrderedDict([('mrr@10', 0.622),
              ('recall@10', 0.1663),
              ('precision@10', 0.3115)]),
 'test_result': OrderedDict([('mrr@10', 0.3999),
              ('recall@10', 0.2048),
              ('precision@10', 0.1351)])}

### Inference

In [2]:
# model, dataset 불러오기
model_path = './saved/EASE-Nov-23-2024_10-54-23.pth'
config, model, dataset, train_data, valid_data, test_data = load_data_and_model(model_path)
    
# device 설정
device = config.final_config_dict['device']

d:\Program\Dev\ANACONDA\envs\movie\lib\site-packages\recbole\quick_start\quick_start.py:250: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_file

In [15]:
# user, item id -> token 변환 array
user_id2token = dataset.field2id_token['user_id']
item_id2token = dataset.field2id_token['item_id']
    
# user-item sparse matrix
matrix = dataset.inter_matrix(form='csr')

In [16]:
# user id, predict item id, score 저장 변수 
pred_list = None
user_list = None
score_list = None

model.eval()
for data in test_data:
    interaction = data[0].to(device)
    score = model.full_sort_predict(interaction)
    
    rating_pred = score.cpu().data.numpy().copy().flatten()  # 1차원 배열로 변환
    user_id = interaction['user_id'].cpu().item()  # 스칼라 값으로 변환
    
    # 사용자가 상호작용한 아이템 인덱스를 가져옵니다
    interacted_indices = matrix[user_id].indices
    
    # 상호작용한 아이템의 점수를 0으로 설정합니다
    rating_pred[interacted_indices] = 0
    
    # 상위 10개 아이템 인덱스 추출
    ind = np.argpartition(rating_pred, -20)[-20:]
    arr_ind = rating_pred[ind]
    
    # 추출된 값들을 내림차순으로 정렬하기 위한 인덱스를 얻음
    arr_ind_argsort = np.argsort(arr_ind)[::-1]
    
    # 실제 값들을 정렬된 순서대로 인덱스 배열에 적용
    batch_pred_list = ind[arr_ind_argsort]
    batch_score_list = arr_ind[arr_ind_argsort]
    
    # 예측값 저장
    if pred_list is None:
        pred_list = batch_pred_list
        score_list = batch_score_list
        # batch_pred_list 길이만큼 user_id를 반복
        user_list = np.repeat(user_id, len(batch_pred_list))
    else:
        pred_list = np.append(pred_list, batch_pred_list, axis=0)
        score_list = np.append(score_list, batch_score_list, axis=0)
        # batch_pred_list 길이만큼 user_id를 반복하여 추가
        user_list = np.append(user_list, np.repeat(user_id, len(batch_pred_list)), axis=0)


In [17]:
# 결과를 저장할 빈 리스트 초기화
final_result = []

# user_list, pred_list, score_list에 있는 값들을 함께 저장
for user, item, score in zip(user_list, pred_list, score_list):
    # user_id2token을 사용하여 변환된 사용자 ID를 얻고
    original_user_seq = user_id2token[user]

    # item_id2token을 사용하여 변환된 아이템 ID를 얻고 
    original_item_seq = item_id2token[item]

    # 사용자 ID, 아이템 ID, 예측 점수를 튜플로 저장
    final_result.append((original_user_seq, original_item_seq, score))

final_result


[('11', '4886', 0.8268838),
 ('11', '4370', 0.7548243),
 ('11', '8961', 0.7517703),
 ('11', '40815', 0.69971085),
 ('11', '47', 0.54835236),
 ('11', '3996', 0.51813924),
 ('11', '1704', 0.51013464),
 ('11', '32587', 0.4920214),
 ('11', '1127', 0.47097677),
 ('11', '7373', 0.4667867),
 ('11', '8360', 0.45052454),
 ('11', '3081', 0.44645295),
 ('11', '2', 0.44508183),
 ('11', '7438', 0.43942633),
 ('11', '2174', 0.43931624),
 ('11', '37386', 0.42387974),
 ('11', '590', 0.42279437),
 ('11', '1206', 0.4143572),
 ('11', '37729', 0.40597358),
 ('11', '1682', 0.39908507),
 ('14', '919', 0.53935236),
 ('14', '1223', 0.4934494),
 ('14', '1035', 0.44524604),
 ('14', '1907', 0.43798834),
 ('14', '6539', 0.43737197),
 ('14', '1198', 0.4330997),
 ('14', '1028', 0.4219346),
 ('14', '2011', 0.41975918),
 ('14', '7147', 0.40701962),
 ('14', '5816', 0.4038603),
 ('14', '1073', 0.39745927),
 ('14', '4016', 0.3853792),
 ('14', '8368', 0.37522268),
 ('14', '4857', 0.36862877),
 ('14', '914', 0.36260253),


In [18]:
# 결과를 DataFrame으로 변환하고 CSV 파일로 저장
final_dataframe = pd.DataFrame(final_result, columns=['user', 'item', 'score'])
# final_dataframe.to_csv('./data/eval/recbole_ease_1.csv', index=False)
final_dataframe.to_csv('./data/eval/score/score_ease_1.csv', index=False)
final_dataframe

,user,item,score
0,11,4886,0.826884
1,11,4370,0.754824
2,11,8961,0.751770
3,11,40815,0.699711
4,11,47,0.548352
...,...,...,...
627195,138493,597,0.419247
627196,138493,589,0.418782
627197,138493,8961,0.415376
627198,138493,45517,0.407788
